In [20]:
# Import library
import time
import datetime
from random import randint
from kafka3 import KafkaProducer
import pandas as pd
import pymongo
from pymongo import MongoClient
import json

In [21]:
# Load AQUA streaming data
aqua_streaming = pd.read_csv('../Datasets/hotspot_AQUA_streaming.csv')

In [22]:
# Change the ip address to host ip address
ip_address = "192.168.224.1"

In [23]:
# Get Latest Climate Date
client = MongoClient(ip_address, 27017) 
db = client['A3_db'] # Using A3_db
fire_collection = db['fire_historic'] # Using collection fire_historic
response = fire_collection.find({},{'date':1}).sort('date',-1).limit(1) # Get latest document
latest_time = response[0]['date']

In [24]:
# Function to turn string into ISO date
def date_to_iso(date, date_format='%d/%m/%Y'):
    return datetime.strptime(str(date), date_format)

# Map function to value, if error is raised return None
# If value is classified as null also return None
def val_or_null(value, func, null=None):
    try:
        value = func(value)
        if value == null:
            return None
        else:
            return value
    except:
        return None

In [25]:
def publish_message(producer_instance, topic_name, message):
    try:
        data = message.encode('utf-8')
        producer_instance.send(topic_name, data)
        producer_instance.flush()
        print('Message published successfully. Data:\n ' + str(data) + '\n')
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

In [26]:
# Create Kafka producer
producer = KafkaProducer(bootstrap_servers=f'{ip_address}:9092', 
                         api_version=(0, 10))

# We will make the data being send every 1/5 days + some random minutes (within 1/5 days)
# This to ensure the hotspot data being send is scattered across different hours
minute_window = (60*24)/5 # Number of minutes in a day / number of time hotspot data being daily

event_time = latest_time

# Iterate over the AQUA data and send messages to Kafka topic
while True:
    # Select row with replacement
    row = aqua_streaming.sample(n=1, replace=True) 
    
    # Increment created_time event_time (1/5 day) + some random minutes
    created_time = pd.to_datetime(event_time) + datetime.timedelta(minutes=randint(0,minute_window))
    created_date = created_time.strftime("%Y/%m/%d") # Convert date to string
    created_time = created_time.strftime("%Y/%m/%d %H:%M:%S") # Convert datetime to string
    event_time = event_time + datetime.timedelta(days=1/5) # Increment event_time by 1/5 day 
    
    # Get the value for everything. Get null if value is invalid
    latitude = val_or_null(row.latitude, float)
    longitude = val_or_null(row.longitude, float)
    confidence = val_or_null(row.confidence, float)
    surface_temperature = val_or_null(row.surface_temperature_celcius, float)

    # Produce the document
    document = {
        'date': created_date,
        'datetime': created_time,
        'latitude': latitude,
        'longitude': longitude,
        'confidence': confidence,
        'surface_temperature': surface_temperature,
        'producer': 'Producer2'
    }
    
    # Convert dict (document) into json
    message = json.dumps(document)
    
    # Send message to Kafka topic
    publish_message(producer, 'hotspot_topic', message)
    
    # Sleep for 2 seconds (1/5 day)
    time.sleep(2)

Message published successfully. Data:
 b'{"date": "2023/01/01", "datetime": "2023/01/01 00:17:00", "latitude": -36.3591, "longitude": 141.0682, "confidence": 86.0, "surface_temperature": 61.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/01/01", "datetime": "2023/01/01 06:25:00", "latitude": -36.995, "longitude": 148.268, "confidence": 56.0, "surface_temperature": 37.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/01/01", "datetime": "2023/01/01 13:17:00", "latitude": -36.7338, "longitude": 144.5479, "confidence": 66.0, "surface_temperature": 43.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/01/01", "datetime": "2023/01/01 17:51:00", "latitude": -34.8919, "longitude": 142.0529, "confidence": 75.0, "surface_temperature": 49.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/01/01", "datetime": "2023/01/01 23:40:00", "latitude": -36.1532, "

Message published successfully. Data:
 b'{"date": "2023/01/08", "datetime": "2023/01/08 17:01:00", "latitude": -36.934, "longitude": 142.5759, "confidence": 64.0, "surface_temperature": 47.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/01/08", "datetime": "2023/01/08 22:41:00", "latitude": -36.163, "longitude": 145.98, "confidence": 79.0, "surface_temperature": 53.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/01/09", "datetime": "2023/01/09 00:25:00", "latitude": -36.8948, "longitude": 142.3282, "confidence": 76.0, "surface_temperature": 49.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/01/09", "datetime": "2023/01/09 05:33:00", "latitude": -36.7287, "longitude": 142.6017, "confidence": 99.0, "surface_temperature": 85.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/01/09", "datetime": "2023/01/09 13:48:00", "latitude": -38.0631, "lo

Message published successfully. Data:
 b'{"date": "2023/01/16", "datetime": "2023/01/16 07:23:00", "latitude": -35.9326, "longitude": 143.9772, "confidence": 57.0, "surface_temperature": 42.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/01/16", "datetime": "2023/01/16 12:04:00", "latitude": -36.8373, "longitude": 146.1811, "confidence": 100.0, "surface_temperature": 50.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/01/16", "datetime": "2023/01/16 17:23:00", "latitude": -36.9826, "longitude": 143.6671, "confidence": 78.0, "surface_temperature": 51.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/01/16", "datetime": "2023/01/16 20:58:00", "latitude": -37.415, "longitude": 148.105, "confidence": 100.0, "surface_temperature": 52.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/01/17", "datetime": "2023/01/17 01:46:00", "latitude": -36.6686,

Message published successfully. Data:
 b'{"date": "2023/01/23", "datetime": "2023/01/23 22:34:00", "latitude": -38.0401, "longitude": 143.9388, "confidence": 87.0, "surface_temperature": 61.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/01/24", "datetime": "2023/01/24 03:27:00", "latitude": -36.8835, "longitude": 142.2098, "confidence": 90.0, "surface_temperature": 67.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/01/24", "datetime": "2023/01/24 06:58:00", "latitude": -34.395, "longitude": 142.0522, "confidence": 80.0, "surface_temperature": 54.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/01/24", "datetime": "2023/01/24 10:31:00", "latitude": -36.7317, "longitude": 142.0162, "confidence": 64.0, "surface_temperature": 42.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/01/24", "datetime": "2023/01/24 16:00:00", "latitude": -38.038, "

Message published successfully. Data:
 b'{"date": "2023/01/31", "datetime": "2023/01/31 10:56:00", "latitude": -35.4616, "longitude": 143.7251, "confidence": 80.0, "surface_temperature": 53.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/01/31", "datetime": "2023/01/31 18:53:00", "latitude": -36.3638, "longitude": 143.9086, "confidence": 71.0, "surface_temperature": 46.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/01/31", "datetime": "2023/01/31 19:58:00", "latitude": -36.6803, "longitude": 141.0957, "confidence": 60.0, "surface_temperature": 44.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/02/01", "datetime": "2023/02/01 03:43:00", "latitude": -35.6469, "longitude": 142.376, "confidence": 100.0, "surface_temperature": 93.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/02/01", "datetime": "2023/02/01 06:29:00", "latitude": -38.0643,

...

Message published successfully. Data:
 b'{"date": "2023/03/02", "datetime": "2023/03/02 21:59:00", "latitude": -36.2882, "longitude": 141.1821, "confidence": 99.0, "surface_temperature": 86.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/03/03", "datetime": "2023/03/03 03:02:00", "latitude": -36.2761, "longitude": 145.8256, "confidence": 53.0, "surface_temperature": 43.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/03/03", "datetime": "2023/03/03 07:14:00", "latitude": -36.1569, "longitude": 145.6554, "confidence": 63.0, "surface_temperature": 41.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/03/03", "datetime": "2023/03/03 13:47:00", "latitude": -36.7185, "longitude": 144.5427, "confidence": 81.0, "surface_temperature": 55.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/03/03", "datetime": "2023/03/03 17:44:00", "latitude": -36.7507,

Message published successfully. Data:
 b'{"date": "2023/03/10", "datetime": "2023/03/10 11:22:00", "latitude": -38.527, "longitude": 143.493, "confidence": 64.0, "surface_temperature": 45.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/03/10", "datetime": "2023/03/10 16:34:00", "latitude": -38.4888, "longitude": 146.5226, "confidence": 89.0, "surface_temperature": 64.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/03/10", "datetime": "2023/03/10 23:15:00", "latitude": -37.9047, "longitude": 141.0945, "confidence": 100.0, "surface_temperature": 90.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/03/11", "datetime": "2023/03/11 03:06:00", "latitude": -34.3526, "longitude": 141.6458, "confidence": 98.0, "surface_temperature": 82.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/03/11", "datetime": "2023/03/11 08:56:00", "latitude": -35.0691, 

Message published successfully. Data:
 b'{"date": "2023/03/18", "datetime": "2023/03/18 04:32:00", "latitude": -37.7787, "longitude": 143.1494, "confidence": 68.0, "surface_temperature": 44.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/03/18", "datetime": "2023/03/18 07:40:00", "latitude": -36.4025, "longitude": 142.5598, "confidence": 83.0, "surface_temperature": 57.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/03/18", "datetime": "2023/03/18 11:31:00", "latitude": -36.1104, "longitude": 145.9829, "confidence": 90.0, "surface_temperature": 76.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/03/18", "datetime": "2023/03/18 17:48:00", "latitude": -37.7074, "longitude": 143.3065, "confidence": 82.0, "surface_temperature": 56.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/03/18", "datetime": "2023/03/18 22:03:00", "latitude": -36.6511,

Message published successfully. Data:
 b'{"date": "2023/03/25", "datetime": "2023/03/25 15:51:00", "latitude": -36.738, "longitude": 142.1177, "confidence": 95.0, "surface_temperature": 77.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/03/25", "datetime": "2023/03/25 19:31:00", "latitude": -37.491, "longitude": 141.936, "confidence": 54.0, "surface_temperature": 40.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/03/26", "datetime": "2023/03/26 00:37:00", "latitude": -36.4622, "longitude": 142.4773, "confidence": 83.0, "surface_temperature": 57.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/03/26", "datetime": "2023/03/26 08:43:00", "latitude": -37.956, "longitude": 143.5461, "confidence": 77.0, "surface_temperature": 50.0, "producer": "Producer2"}'

Message published successfully. Data:
 b'{"date": "2023/03/26", "datetime": "2023/03/26 14:10:00", "latitude": -37.9378, "lo

KeyboardInterrupt: 